### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

### Описание данных

* train.csv - данные для обучения
* test.csv - данные для подготовки самбита и проверки
* sampleSubmission.csv - пример корректного но бесполезного сабмита
* other.csv - необязательные данные для доп.статистик и прочих извращений (например обучение word2vec-а)

### Описание полей

* id - внутренний идетификатор
* name - название вакансии
* description - текст вакансии
* target - класс заинтересованности

In [1]:
%%time

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, \
                        accuracy_score, classification_report, roc_auc_score, roc_curve

CPU times: user 482 ms, sys: 154 ms, total: 636 ms
Wall time: 830 ms


## Шаг 1. Импортируем все в Pandas


In [2]:
%%time
sample_submission = pd.read_csv('sampleSubmission.csv',',')
train_df = pd.read_csv('train.csv','\t')
test_df = pd.read_csv('test.csv','\t')
other_df = pd.read_csv('other.csv','\t')

CPU times: user 18 s, sys: 1.8 s, total: 19.8 s
Wall time: 19.8 s


In [3]:
train_df['concat'] = train_df['name']+' '+ train_df['description']
train_df.head()

,id,name,description,target,concat
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1,Заведующий отделом/секцией в магазин YORK (Уру...
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0,Наладчик станков и манипуляторов с ПУ Обязанно...
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0,Разработчик С++ (Криптограф) <strong>Требовани...
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0,Фрезеровщик <p>Условия:</p> <ul> <li>На работу...
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1,Мерчендайзер/продавец-консультант <p><strong>К...


## Шаг 2. Чистим данные и подготавливаем текст

In [4]:
# Подключаем БьютифулСуп
from bs4 import BeautifulSoup             

# Прогоняем отдельное описание через БьютифулСуп   
example1 = BeautifulSoup(train_df['concat'][0], 'html5lib')  

In [5]:
#Подключаем регулярки, чтобы очистить текст от тегов html
import re
# Делаем найти и заменить на ничто для всего что не похоже на регулярку
letters_only = re.sub("[^а-яА-Яa-zA-Z]",           # находим всё что не
                      " ",                   # Меняем на пустоту
                      example1.get_text() )  
print (letters_only)

Заведующий отделом секцией в магазин YORK  Уручье  В НОВЫЙ МАГАЗИН YORK  хозтовары  приглашаем на постоянную работу руководителя секции  хозтовары   Обязанности      организация эффективного и культурного обслуживания покупателей организация приемки сдачи товаров на склад консультация покупателей по вопросам  касающимся оказываемых услуг контроль своевременной подачи товаров в торговую секцию проверка качества  сроков годности ТМЦ  наличие маркировок  ценников на товарах организация бесперебойной работы товарной секции и участка склада  закрепленного за ней контроль за сохранностью товаров  торгового оборудования и прочих материальных ценностей инвентаризация товаров    Требования   знание товарной группы знание правил приемки товара опыт работы в розничной торговле в качестве продавца   р    заместителя заведующего секцией отделом ответственность  лидерские качества  опыт управления коллективом      Условия   НОВЫЙ современный магазин розничной торговли хозтоварами хороший коллектив д

In [26]:
lower_case = letters_only.lower()
words1 = lower_case.split()  

In [27]:
import nltk
import pymystem3
mystem = pymystem3.Mystem()
#nltk.download()  # Если NLTK не установлен, то надо раскомментровать этот блок и исполнить

In [28]:
from nltk.corpus import stopwords # Импортируем стоп-слова русскго языка
print( stopwords.words('russian') )

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [29]:
%%time
import time
# Удаляем стоп-слова русского языка из текста words
a = time.clock()
words = [w for w in words1 if not w in stopwords.words('russian')]
b = time.clock()
print('Заняло {} сек'.format(b-a))
print (words)
# Посмоторим что получится, если прогнать мистемом. 
a = time.clock()
words = [mystem.lemmatize(w) for w in words1 if not w in stopwords.words('russian')]
b = time.clock()
print('Заняло {} сек'.format(b-a))
print (words)

Заняло 0.02363099999999818 сек
['заведующий', 'отделом', 'секцией', 'магазин', 'york', 'уручье', 'новый', 'магазин', 'york', 'хозтовары', 'приглашаем', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары', 'обязанности', 'организация', 'эффективного', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'склад', 'консультация', 'покупателей', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'участка', 'склада', 'закрепленного', 'контроль', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'розничной', 'торговле', 'качестве', 'про

In [39]:
temp = ''
for i in words:
    temp += (i[0]+' ')
temp

'заведующий отдел секция магазин york уручье новый магазин york хозтовары приглашать постоянный работа руководитель секция хозтовары обязанность организация эффективный культурный обслуживание покупатель организация приемка сдача товар склад консультация покупатель вопрос касаться оказывать услуга контроль своевременный подача товар торговый секция проверка качество срок годность тмц наличие маркировка ценник товар организация бесперебойный работа товарный секция участок склад закреплять контроль сохранность товар торговый оборудование прочий материальный ценность инвентаризация товар требование знание товарный группа знание правило приемка товар опыт работа розничный торговля качество продавец р заместитель заведующий секция отдел ответственность лидерский качество опыт управление коллектив условие новый современный магазин розничный торговля хозтовары хороший коллектив достойный зарплата премия соц пакет график работа пн пт плавать место работа ул никифоров уручье '

In [40]:
def vac_to_words( description, Mystem = True ):
    """
    Функция делает преобразование поля description из формата html в формат с отдельными значащими словами
    На вход передается  описание и параметр True/False. Если True, то работает стеммер, иначе нет
    """
    description_text = BeautifulSoup(description, 'html5lib').get_text() #получаем текст      
    letters_only = re.sub('[^а-яА-Яa-zA-Z]', ' ', description_text) #удаляем всё что не буквы
    words = letters_only.lower().split() #получаем слова в нижнем регистре                             
    stops = set(stopwords.words('russian'))  # формируем set стоп слов русского языка                  
    if Mystem: #удаляем минус фразы и  оставляем либо "стеммы", либо значащие слова
        meaningful_words = [mystem.lemmatize(w) for w in words if not w in stops]
        temp = ''
        for i in meaningful_words:
            temp += (i[0]+' ')
        return temp
    else: 
        meaningful_words = [w for w in words if not w in stops]
        return( ' '.join( meaningful_words ))  # собираем слова в строку с разделителеим пробел 

In [41]:
clean_vac = vac_to_words( train_df['concat'][0])
print(clean_vac)

заведующий отдел секция магазин york уручье новый магазин york хозтовары приглашать постоянный работа руководитель секция хозтовары обязанность организация эффективный культурный обслуживание покупатель организация приемка сдача товар склад консультация покупатель вопрос касаться оказывать услуга контроль своевременный подача товар торговый секция проверка качество срок годность тмц наличие маркировка ценник товар организация бесперебойный работа товарный секция участок склад закреплять контроль сохранность товар торговый оборудование прочий материальный ценность инвентаризация товар требование знание товарный группа знание правило приемка товар опыт работа розничный торговля качество продавец р заместитель заведующий секция отдел ответственность лидерский качество опыт управление коллектив условие новый современный магазин розничный торговля хозтовары хороший коллектив достойный зарплата премия соц пакет график работа пн пт плавать место работа ул никифоров уручье 


In [42]:
# Посмотрим сколько всего описаний есть в train_df
num_vacs = train_df['concat'].size
num_vacs

200000

In [43]:
%%time
clean_train_vacs = [] #делаем пустой лист для записи всех описаний с отдельными словами

print ('Очищаем и собираем нормализованные описания из train_df...\n')

# Пройдем в цикле по всем описаниям
for i in range( 0, num_vacs ):
    
    # Для каждого описания вызываем функцию нормировки текста описания и добавляем нормированный текст в лист
    
    if( (i+1)%10000 == 0 ): #чтобы не было скучно, каждые 10 000 описаний будем печатать развлекательный текст
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_vacs,(i+1)/num_vacs )) 
        
    clean_train_vacs.append( vac_to_words( train_df['concat'][i]) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из train_df...

Описание 10000 из 200000 в обработке... 5%
Описание 20000 из 200000 в обработке... 10%
Описание 30000 из 200000 в обработке... 15%
Описание 40000 из 200000 в обработке... 20%
Описание 50000 из 200000 в обработке... 25%
Описание 60000 из 200000 в обработке... 30%
Описание 70000 из 200000 в обработке... 35%
Описание 80000 из 200000 в обработке... 40%
Описание 90000 из 200000 в обработке... 45%
Описание 100000 из 200000 в обработке... 50%
Описание 110000 из 200000 в обработке... 55%
Описание 120000 из 200000 в обработке... 60%
Описание 130000 из 200000 в обработке... 65%
Описание 140000 из 200000 в обработке... 70%
Описание 150000 из 200000 в обработке... 75%
Описание 160000 из 200000 в обработке... 80%
Описание 170000 из 200000 в обработке... 85%
Описание 180000 из 200000 в обработке... 90%
Описание 190000 из 200000 в обработке... 95%
Описание 200000 из 200000 в обработке... 100%
Я закончила!!!
CPU times: user 20min 11s, sys: 3m

## Создаём свойства через Bag of Words

In [44]:
%%time
print ('Фигачим Bag of words...\n')
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 10000) 


# 5 000 фич считались 20.1 сек
# прогоняем полученный в прошлом пункте лист с текстами через fit_transform() 
train_data_features = vectorizer.fit_transform(clean_train_vacs)

# Переделаем полученные свойства в array 
train_data_features = train_data_features.toarray()

Фигачим Bag of words...

CPU times: user 19.4 s, sys: 5.34 s, total: 24.7 s
Wall time: 25.3 s


In [45]:
train_data_features.shape

(200000, 10000)

In [46]:
# Посмотрим на получившийся словарь 
vocab = vectorizer.get_feature_names()
print (vocab)

['aa', 'abap', 'abb', 'abbyy', 'abc', 'abi', 'abilities', 'ability', 'able', 'about', 'above', 'abroad', 'acceptance', 'access', 'accordance', 'according', 'account', 'accountable', 'accounting', 'accounts', 'accuracy', 'accurate', 'achieve', 'achieving', 'acoola', 'acquisition', 'acrobat', 'acronis', 'across', 'act', 'action', 'actions', 'active', 'actively', 'activities', 'activity', 'ad', 'adapt', 'add', 'addition', 'additional', 'address', 'adidas', 'administration', 'administrator', 'ado', 'adobe', 'ads', 'advance', 'advanced', 'advantage', 'advertising', 'adwords', 'affiliate', 'after', 'again', 'against', 'age', 'agencies', 'agency', 'agent', 'agile', 'agreed', 'ai', 'air', 'ajax', 'akka', 'al', 'algorithms', 'align', 'all', 'allowance', 'allows', 'along', 'already', 'also', 'altium', 'always', 'am', 'amazing', 'amazon', 'ambitious', 'america', 'american', 'amocrm', 'among', 'amount', 'an', 'analysis', 'analyst', 'analysts', 'analytic', 'analytical', 'analytics', 'analyze', 'ana

In [47]:
import numpy as np

# Просуммируем счетчики
dist = np.sum(train_data_features, axis=0)

# Посмотрим сколько раз каждое слово встречается в train_df

for tag, count in zip(vocab, dist):
    print (count, tag)

149 aa
530 abap
161 abb
172 abbyy
106 abc
112 abi
191 abilities
3514 ability
1019 able
1732 about
233 above
136 abroad
144 acceptance
513 access
232 accordance
654 according
1314 account
121 accountable
133 accounting
322 accounts
87 accuracy
136 accurate
374 achieve
115 achieving
142 acoola
157 acquisition
117 acrobat
208 acronis
917 across
190 act
194 action
209 actions
1097 active
204 actively
1420 activities
241 activity
1331 ad
151 adapt
83 add
97 addition
428 additional
178 address
129 adidas
608 administration
287 administrator
137 ado
2434 adobe
117 ads
257 advance
992 advanced
698 advantage
332 advertising
1852 adwords
197 affiliate
342 after
183 again
118 against
122 age
184 agencies
215 agency
92 agent
2811 agile
138 agreed
238 ai
88 air
1249 ajax
141 akka
103 al
299 algorithms
88 align
2975 all
104 allowance
109 allows
116 along
117 already
771 also
240 altium
316 always
139 am
141 amazing
519 amazon
205 ambitious
105 america
167 american
306 amocrm
416 among
210 amount
429

97 her
101 here
141 hh
145 hi
627 hibernate
2664 high
608 higher
138 highest
392 highload
778 highly
87 hilfiger
123 hilti
113 hire
187 hiring
176 his
121 history
129 hive
93 hmi
117 hoc
197 hoff
93 holding
140 holidays
1871 home
1278 horeca
135 hosting
150 hotel
110 hour
930 hours
374 house
666 how
97 however
859 hp
118 hq
1143 hr
7858 html
1680 http
696 https
292 huawei
184 huge
159 human
322 hundreds
97 hybris
558 hyper
141 hyundai
116 iaas
963 ibm
87 ibs
260 ico
154 id
179 ide
225 idea
205 ideal
89 ideally
747 ideas
121 idef
86 identification
539 identify
193 identifying
135 identity
110 ids
102 ie
1370 if
180 ii
149 iii
411 iis
85 ikea
1273 illustrator
114 image
113 imaginarium
120 imagine
355 impact
180 imperial
760 implement
1327 implementation
85 implemented
398 implementing
523 important
757 improve
525 improvement
225 improvements
198 improving
32114 in
260 inc
249 incanto
107 incident
150 incidents
204 incity
457 include
285 includes
1762 including
217 increase
112 increasin

258 samsung
281 san
4605 sap
358 sas
756 sass
110 sat
147 satisfaction
348 scada
559 scala
106 scalability
274 scalable
383 scale
101 sccm
82 scenarios
648 schedule
95 schedules
293 schneider
163 school
1056 science
149 scientist
85 scm
84 scom
286 scope
132 scratch
318 script
479 scripting
192 scripts
1989 scrum
246 scss
291 sd
639 sdk
115 sdlc
438 se
516 search
120 searching
309 secret
138 sector
85 sectors
190 secure
1356 security
180 see
255 seeing
246 seeking
145 segment
97 sela
125 selection
708 selenium
1188 self
361 sell
329 selling
137 sem
125 seminars
246 send
3703 senior
213 sense
3839 seo
222 series
90 serm
152 serve
4670 server
329 servers
2585 service
158 servicedesk
3773 services
100 serving
124 servlets
83 sessions
512 set
88 sets
251 setting
153 setup
275 several
103 shape
373 share
96 shared
797 sharepoint
316 sharing
546 shell
330 shop
96 shopping
161 short
620 should
163 show
102 si
541 sick
555 side
112 siebel
125 siem
642 siemens
148 significant
257 signs
158 sili

199 взаимосвязь
525 взамен
207 взвешивание
141 взвешивать
696 взгляд
119 взнос
398 взрослый
153 взыскание
1569 взять
210 ви
82 вибрация
11394 вид
401 видение
3741 видео
86 видеокамера
1264 видеонаблюдение
200 видеоролик
123 видеосъемка
4176 видеть
83 виджет
111 видимость
211 видный
192 виза
250 визажист
1698 визит
312 визитка
1709 визитный
940 визуализация
1509 визуальный
126 вика
87 викладка
325 виконання
152 використання
102 виктория
88 вилка
160 вимог
869 вимога
287 винный
445 вино
129 виноградный
92 вип
116 виробництва
87 вирощування
1163 виртуализация
997 виртуальный
91 вирусный
348 висок
119 високий
111 витебск
2497 витрина
86 вих
403 вищ
465 вк
859 вклад
252 вкладывать
114 вкладываться
3177 включать
192 включаться
4174 включая
303 включение
669 вконтакте
116 вкс
1407 вкус
181 вкусно
85 вкусности
1421 вкусный
124 вкусняшка
215 вл
689 владелец
17407 владение
2220 владеть
281 владивосток
291 владимир
349 владимирский
409 власть
398 вливаться
884 влияние
2316 влиять
86 влксм
131 вло

824 имя
91 иначе
9610 инвентаризация
1713 инвентаризацияхтребование
606 инвентарь
110 инвест
106 инвестировать
744 инвестиционный
575 инвестиция
344 инвестор
560 ингредиент
318 индекс
460 индексация
152 индексировать
5219 индивидуально
146 индивидуальность
6448 индивидуальный
85 индикатор
112 индия
95 индукционный
477 индустриальный
3346 индустрия
19480 инженер
110 инженерия
4101 инженерный
226 инжиниринг
300 инжиниринговый
1550 инициатива
7271 инициативность
2141 инициативный
290 инициация
407 инициирование
203 инициировать
1599 инкассация
89 инновационность
3814 инновационный
741 инновация
180 иннополис
2693 иногородний
2682 иной
2993 иностранный
95 инсайт
106 инспектировать
263 инспектор
140 инспекционный
292 инспекция
318 инстаграм
84 инстаграмма
372 инсталляция
884 институт
1126 инструктаж
5934 инструкция
11518 инструмент
378 инструментальный
106 инструментальщик
529 инструментарий
1050 интегратор
1582 интеграционный
5221 интеграция
602 интегрировать
378 интеллект
655 интеллектуал

557 некоторые
97 некрасова
266 нематериальный
951 немецкий
568 немного
363 немой
1430 ненормированный
132 необх
3294 необходимо
4239 необходимость
14276 необходимый
281 необычный
187 необязательно
275 необязательный
1961 неограниченный
1357 неоконченный
127 неопределенность
129 неординарный
168 неотъемлемый
85 неоценимый
259 неповторимый
504 неполадка
45161 неполный
719 непосредственно
1936 непосредственный
96 непреодолимый
140 непрерывность
1544 непрерывный
942 непродовольственный
126 непростой
106 нерабочий
222 неравнодушный
300 неразрушающий
211 нержавеющий
130 несанкционированный
4832 несколько
248 несложный
173 несмотря
664 несоответствие
247 несоответствующий
144 нестандартно
3056 нестандартный
1045 нести
120 нестль
1485 несчастный
123 нетерпение
457 нетиповой
469 нетривиальный
159 нетрудоспособность
316 неформальный
132 нефтебаза
583 нефтегазовый
88 нефтепереработка
952 нефтепродукт
229 нефтехимический
124 нефтехимия
355 нефть
273 нефтяной
99 нештатный
119 нженер
11017 ниже
489 

457 посадочный
124 посвящать
273 посев
173 посевная
504 поселок
1733 посетитель
452 посещаемость
1277 посещать
6473 посещение
141 поскольку
2920 последний
402 последовательность
131 последовательный
221 последствие
2067 последующий
209 послепродажный
547 послуга
168 посменно
427 посменный
313 посмотреть
346 пособие
85 посредник
92 посредственность
999 посредством
1307 пост
9247 поставка
9671 поставлять
10489 поставщик
5732 постановка
260 постановление
110 постараться
313 постельный
128 постинг
93 посторонний
4936 постоянно
15288 постоянный
85 постпечатный
138 постпродажный
8289 построение
1904 построить
2650 поступать
1480 поступление
926 посуда
103 посылка
2497 потенциал
129 потенциально
7917 потенциальный
4327 потеря
2474 поток
4515 потолок
98 поточный
111 потр
3699 потребитель
1457 потребительский
856 потребление
91 потреблять
7885 потребность
650 потребоваться
138 потрясать
240 поучаствовать
120 поучиться
200 поход
239 похожий
796 почасовой
228 почва
1568 почему
1283 почитать
6440 

11207 рубль
110 руда
160 рудник
105 руза
4603 рука
22963 руководитель
1031 руководить
15525 руководство
348 руководствоваться
4148 руководящий
94 рукоделие
82 рулонный
367 рум
363 румянцевый
395 рунет
4219 русский
178 русый
101 русь
98 рутина
339 рутинный
113 ручка
1423 ручной
95651 рф
135 рц
429 рыба
96 рыбалка
143 рыбацкий
363 рыбный
41252 рынок
867 рыночный
381 рэа
1812 ряд
260 рядовой
31471 рядом
297 рязанский
326 рязань
191 сrm
410 савеловский
837 сад
494 садовый
21071 сайт
93 саларьево
11244 салон
3182 сам
1261 самара
389 самарский
94 самовывоз
455 самодисциплина
222 самоконтроль
92 самолет
222 самомотивация
213 самообразование
243 самообслуживание
131 самообучаемость
500 самообучение
1781 самоорганизация
1100 самоорганизованность
93 самоотдача
99 самописный
855 саморазвитие
1583 самореализация
292 самосовершенствование
6363 самостоятельно
4282 самостоятельность
4176 самостоятельный
99 самсон
11886 самый
543 сан
292 санаторий
569 санаторный
750 санитария
3329 санитарный
5652 санк

656 товародвижение
4361 товарооборот
166 тоир
174 ток
462 токарная
833 токарный
823 токарь
130 толковый
154 толстой
268 тольятти
362 томск
130 томский
187 тонкий
414 тонкость
188 тонна
372 тоо
2859 топ
707 топливный
1742 топливо
236 топовый
157 топология
93 тор
245 торг
866 торги
146 торговать
14255 торговля
66309 торговый
88 торинг
229 торт
12296 тот
15007 точка
1447 точно
901 точность
727 точный
1903 тп
106 тпа
617 тр
286 травматизм
615 традиционный
382 традиция
304 тракт
163 трактор
97 тракторист
83 тракторный
105 транзакционный
145 транзакция
401 транслировать
222 трансляция
189 транснефть
7139 транспорт
716 транспортировка
3820 транспортный
222 трансфер
259 трансформатор
123 трансформаторный
207 трансформация
185 трасса
115 трассировка
106 трата
365 тратить
2722 трафик
163117 требование
389 требовательность
188 требовательный
3372 требовать
19269 требоваться
2058 трейд
313 трекер
345 трекинг
190 трекинговый
102 трен
108 тренаж
420 тренажер
792 тренажерный
1572 тренд
97 трендовый
1

## Шаг 3. Решам задачу через модель Random Forest

In [48]:
#Делаем трейн тест сплит

from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

X_train, X_test, y_train, y_test = train_test_split(train_data_features, train_df['target'], test_size=0.33, random_state=42)
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


X_train  (134000, 10000)
X_test  (66000, 10000)
y_train  (134000,)
y_test  (66000,)


In [49]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Пока делаем 100 деревьев, но в следующих вариантах домашки попробую сделать 50
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

forest = forest.fit( X_train, y_train )
print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 52min 20s, sys: 1min 26s, total: 53min 47s
Wall time: 6min 33s


In [50]:
predict = forest.predict(X_test)

In [51]:
# прошлый результаты были 0.97 0.97 0.97

print('#'*20,'>>> Random Trees <<<','#'*20,'\n',classification_report(y_test, predict),'\n')

#################### >>> Random Trees <<< #################### 
              precision    recall  f1-score   support

          0       0.97      0.96      0.97     34984
          1       0.96      0.97      0.96     31016

avg / total       0.97      0.97      0.97     66000
 



# Шаг 4. Готовим файл к отправке

In [52]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Делаем random forest на 100 деревьев
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

# В версии с 5 000 фич обучение заняло 22 минут 12 секунд, старт в 23-37
forest = forest.fit( train_data_features, train_df['target'] )

print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 1h 38min 30s, sys: 3min, total: 1h 41min 30s
Wall time: 12min 7s


In [53]:
test_df['concat'] = test_df['name']+' '+ test_df['description']

test_df.shape

(170179, 4)

In [54]:
num_test_vacs = len(test_df['concat'])

In [55]:
%%time
clean_test_vacs = []
print ('Очищаем и собираем нормализованные описания из test_df...\n')
for i in range(0,num_test_vacs):
    if( (i+1) % 10000 == 0 ):
        print ('Описание {} из {} в обработке... {:.0%}'.format( i+1, num_test_vacs, (i+1)/num_test_vacs )) 
    clean_test_vacs.append( vac_to_words( test_df['concat'][i] ) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из test_df...

Описание 10000 из 170179 в обработке... 6%
Описание 20000 из 170179 в обработке... 12%
Описание 30000 из 170179 в обработке... 18%
Описание 40000 из 170179 в обработке... 24%
Описание 50000 из 170179 в обработке... 29%
Описание 60000 из 170179 в обработке... 35%
Описание 70000 из 170179 в обработке... 41%
Описание 80000 из 170179 в обработке... 47%
Описание 90000 из 170179 в обработке... 53%
Описание 100000 из 170179 в обработке... 59%
Описание 110000 из 170179 в обработке... 65%
Описание 120000 из 170179 в обработке... 71%
Описание 130000 из 170179 в обработке... 76%
Описание 140000 из 170179 в обработке... 82%
Описание 150000 из 170179 в обработке... 88%
Описание 160000 из 170179 в обработке... 94%
Описание 170000 из 170179 в обработке... 100%
Я закончила!!!
CPU times: user 15min 32s, sys: 2min 47s, total: 18min 20s
Wall time: 30min 36s


In [56]:
%%time
# Фигачим bag of words по тестовым данным

test_data_features = vectorizer.transform(clean_test_vacs)
test_data_features = test_data_features.toarray()

CPU times: user 16.1 s, sys: 5.09 s, total: 21.2 s
Wall time: 21.4 s


In [57]:
%%time
# Через Random Forest делаем предсказание
result_test = forest.predict(test_data_features)

CPU times: user 44.9 s, sys: 14.2 s, total: 59.1 s
Wall time: 28.1 s


In [58]:
# Копируем результат в датафрейм с полями id и target 
# предыдущий реузльтат был таким http://take.ms/cVRK4
output = pd.DataFrame( data={'id':test_df['id'], 'target':result_test} )
output.head()

,id,target
0,200000,1
1,200001,1
2,200002,1
3,200003,1
4,200004,0


In [59]:
output.to_csv( "Random_Forest_10000mystem-and-names.csv", sep=',', encoding='utf-8',index=False )